In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

# Parameter tuning

For Hyperparameter Tuning we considered GridSearchCV and RandomizedSearchCV. GridSearchCV evaluates all the options listed in the parameter grid, which can be computationally expensive. Therefore, we chose RandomizedSearchCV, which randomly chooses from different parameter combinations to find the best score.

For each dataset, we carry out a parameter tuning for the best combination (highest f1 score) from the baseline. RandomForestClassifier gave the highest f1 score for each dataset, except for the Congressional Voting. In case of the Congressional Voting dataset, DecisionTreeClassifier and RandomForestClassifier both gave the highest f1 scores. For this dataset, we perform a parameter tuning for all 3 classifiers.

In [2]:
combined_results = pd.read_csv('data/combined_results.csv')

## 1. Dataset Congressional Voting

In [3]:
congressional_voting_results = combined_results[combined_results['dataset'] == 'congression_voting']

Top combinations based on F1 score:

In [4]:
congressional_voting_results.nlargest(10, 'f1')

,dataset,classifier,splitting,feature_selection,scaling,n_features_orig,n_features,accuracy,precision,recall,f1,preprocess_time,training_time,prediction_time,f1_std
82,congression_voting,RandomForestClassifier,holdout,none,standard,16,16,0.954545,0.954678,0.954545,0.954380,0.012542,0.084082,0.018900,NaN
88,congression_voting,RandomForestClassifier,holdout,none,robust,16,16,0.954545,0.954678,0.954545,0.954380,0.000000,0.104615,0.000000,NaN
94,congression_voting,RandomForestClassifier,holdout,none,minmax,16,16,0.954545,0.954678,0.954545,0.954380,0.000740,0.088130,0.016628,NaN
100,congression_voting,RandomForestClassifier,holdout,p-value,none,16,14,0.954545,0.954678,0.954545,0.954380,0.032598,0.083973,0.018123,NaN
106,congression_voting,RandomForestClassifier,holdout,p-value,standard,16,14,0.954545,0.954678,0.954545,0.954380,0.038320,0.095003,0.000000,NaN
110,congression_voting,DecisionTreeClassifier,holdout,p-value,robust,16,14,0.954545,0.954678,0.954545,0.954380,0.047630,0.000996,0.000000,NaN
112,congression_voting,RandomForestClassifier,holdout,p-value,robust,16,14,0.954545,0.954678,0.954545,0.954380,0.024520,0.099888,0.000000,NaN
116,congression_voting,DecisionTreeClassifier,holdout,p-value,minmax,16,14,0.954545,0.954678,0.954545,0.954380,0.047163,0.000966,0.000000,NaN
118,congression_voting,RandomForestClassifier,holdout,p-value,minmax,16,14,0.954545,0.954678,0.954545,0.954380,0.029113,0.096108,0.004035,NaN
97,congression_voting,KNeighborsClassifier,cv,p-value,none,16,13,0.953911,0.954145,0.953911,0.953808,0.030002,0.000139,0.003349,0.014932


Best combination per classifier:

In [5]:
top_per_classifier = congressional_voting_results.groupby('classifier').apply(lambda group: group.nlargest(3, 'f1')).reset_index(drop=True)

top_per_classifier

,dataset,classifier,splitting,feature_selection,scaling,n_features_orig,n_features,accuracy,precision,recall,f1,preprocess_time,training_time,prediction_time,f1_std
0,congression_voting,DecisionTreeClassifier,holdout,p-value,robust,16,14,0.954545,0.954678,0.954545,0.954380,0.047630,0.000996,0.000000,NaN
1,congression_voting,DecisionTreeClassifier,holdout,p-value,minmax,16,14,0.954545,0.954678,0.954545,0.954380,0.047163,0.000966,0.000000,NaN
2,congression_voting,DecisionTreeClassifier,cv,p-value,robust,16,13,0.940063,0.940295,0.940063,0.939847,0.035642,0.000000,0.000000,0.011615
3,congression_voting,KNeighborsClassifier,cv,p-value,none,16,13,0.953911,0.954145,0.953911,0.953808,0.030002,0.000139,0.003349,0.014932
4,congression_voting,KNeighborsClassifier,cv,p-value,robust,16,13,0.953911,0.954145,0.953911,0.953808,0.037020,0.000171,0.000147,0.014932
5,congression_voting,KNeighborsClassifier,cv,p-value,minmax,16,13,0.953911,0.954145,0.953911,0.953808,0.033656,0.000000,0.005726,0.014932
6,congression_voting,RandomForestClassifier,holdout,none,standard,16,16,0.954545,0.954678,0.954545,0.954380,0.012542,0.084082,0.018900,NaN
7,congression_voting,RandomForestClassifier,holdout,none,robust,16,16,0.954545,0.954678,0.954545,0.954380,0.000000,0.104615,0.000000,NaN
8,congression_voting,RandomForestClassifier,holdout,none,minmax,16,16,0.954545,0.954678,0.954545,0.954380,0.000740,0.088130,0.016628,NaN


### 1.1. RandomForestClassifier
For parameter tuning we use one of the best combinations:
- feature_selection: None
- scaling: standard

In [6]:
# Read the preprocessed dataset
congressional_voting = pd.read_csv("./preprocessed-datasets/CongressionVoting_prepro.csv")
congressional_voting.set_index('ID', inplace=True)
X = congressional_voting.drop("class", axis=1) 
y = congressional_voting['class'] 
  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=18) 

scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the parameter grid for RandomizedSearchCV
param_grid = { 
    'n_estimators': [25, 50, 100, 150], 
    'max_features': ['sqrt', 'log2', None], 
    'max_depth': [10, 20, 30], 
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
} 

In [7]:
# Perform RandomizedSearchCV
random_search = RandomizedSearchCV(RandomForestClassifier(), param_distributions=param_grid, scoring="f1_weighted") 
random_search.fit(X_train_scaled, y_train) 

RandomizedSearchCV(estimator=RandomForestClassifier(),
                   param_distributions={'max_depth': [10, 20, 30],
                                        'max_features': ['sqrt', 'log2', None],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [25, 50, 100, 150]},
                   scoring='f1_weighted')

In [8]:
cv_results = random_search.cv_results_

results_data = {
    'n_estimators': cv_results['param_n_estimators'],
    'max_features': cv_results['param_max_features'],
    'max_depth': cv_results['param_max_depth'],
    'min_samples_split': cv_results['param_min_samples_split'],
    'min_samples_leaf': cv_results['param_min_samples_leaf'],
    'F1_score': cv_results['mean_test_score'],
    'Runtime': cv_results['mean_fit_time'] + cv_results['mean_score_time']
}


results = pd.DataFrame(data=results_data)
results.sort_values(by=['F1_score'], ascending=False, inplace=True)
results.reset_index(drop=True, inplace=True)
results


,n_estimators,max_features,max_depth,min_samples_split,min_samples_leaf,F1_score,Runtime
0,100,None,20,2,4,0.960873,0.160597
1,150,None,20,5,4,0.960873,0.247255
2,150,None,30,10,4,0.960873,0.250263
3,100,None,30,2,2,0.947508,0.176337
4,100,sqrt,30,2,2,0.940985,0.166722
5,25,None,10,5,1,0.940721,0.068539
6,25,sqrt,10,5,1,0.940594,0.044085
7,25,None,10,10,2,0.940444,0.045693
8,50,log2,10,5,1,0.934297,0.090978
9,100,sqrt,20,5,1,0.934297,0.161988


In [9]:
# Get the best estimator from RandomizedSearchCV
best_estimator = random_search.best_estimator_

# Evaluate the best estimator on the test set
y_pred_test = best_estimator.predict(X_test_scaled)
f1_test = metrics.f1_score(y_test, y_pred_test, average='weighted')

largest = congressional_voting_results.nlargest(1, 'f1')
f1_value = largest['f1'].iloc[0]
print(f"f1 score of the best performing combination without parameter tuning: {f1_value}")
print(f"f1 score of the best estimator from RandomizedSearchCV: {f1_test}")

f1 score of the best performing combination without parameter tuning: 0.9543804053607976
f1 score of the best estimator from RandomizedSearchCV: 0.9696969696969697


One of the best combinations (classifier: RandomForest, feature_selection: None, scaling: standard) for the congressional voting dataset had an f1 score = 0.95438. With RandomizedSearchCV we found the best parameter setting which result in a higher f1 score.  

### 1.2. KNeighborsClassifier

For parameter tuning of the KNeighborsClassifier we use one of its the best combinations:
- feature_selection: p-value
- scaling: None

In [10]:
congressional_voting = pd.read_csv("./preprocessed-datasets/CongressionVoting_prepro.csv")
congressional_voting.set_index('ID', inplace=True)
X = congressional_voting.drop("class", axis=1) 
y = congressional_voting['class'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=18) 

# Feature selection
X_train = X_train.loc[:, X_train.apply(pd.Series.nunique) != 1]
p_values = []
for feature in X_train.columns:
    f, p = chi2(X_train[[feature]], y_train)
    p_values.append(p)

p_values = np.array(p_values).reshape(-1)
p_values = pd.Series(p_values, index=X_train.columns).sort_values()
selected_features = p_values[p_values < 0.05].index

X_train_preprocessed, X_test_preprocessed = X_train[selected_features], X_test[selected_features]

print("Without feature selection: ", X_train.shape[1])
print("With feature selection: ", len(selected_features))

# Define the parameter grid for RandomizedSearchCV
param_grid_knn = {
    'n_neighbors': [1, 5, 10, 50, 100],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'p': [1, 2]
}

Without feature selection:  16
With feature selection:  14


In [11]:
# Perform RandomizedSearchCV
random_search = RandomizedSearchCV(KNeighborsClassifier(), param_distributions=param_grid_knn, scoring="f1_weighted") 
random_search.fit(X_train_preprocessed, y_train) 

RandomizedSearchCV(estimator=KNeighborsClassifier(),
                   param_distributions={'algorithm': ['auto', 'ball_tree',
                                                      'kd_tree', 'brute'],
                                        'n_neighbors': [1, 5, 10, 50, 100],
                                        'p': [1, 2],
                                        'weights': ['uniform', 'distance']},
                   scoring='f1_weighted')

In [12]:
cv_results = random_search.cv_results_

results_data = {
    'n_neighbors': cv_results['param_n_neighbors'],
    'weights': cv_results['param_weights'],
    'algorithm': cv_results['param_algorithm'],
    'p': cv_results['param_p'],
    'F1_score': cv_results['mean_test_score'],
    'Runtime': cv_results['mean_fit_time'] + cv_results['mean_score_time']
}


results = pd.DataFrame(data=results_data)
results.sort_values(by=['F1_score'], ascending=False, inplace=True)
results.reset_index(drop=True, inplace=True)
results

,n_neighbors,weights,algorithm,p,F1_score,Runtime
0,1,distance,auto,1,0.946949,0.007003
1,1,uniform,kd_tree,2,0.946949,0.008909
2,1,distance,ball_tree,2,0.939768,0.009107
3,5,uniform,ball_tree,1,0.933708,0.025958
4,5,distance,brute,1,0.933605,0.009279
5,50,uniform,auto,1,0.914782,0.014542
6,50,uniform,brute,1,0.901437,0.009793
7,100,distance,brute,2,0.897608,0.024118
8,100,distance,ball_tree,2,0.897608,0.007088
9,100,uniform,brute,2,0.502562,0.010745


In [13]:
# Get the best estimator from RandomizedSearchCV
best_estimator = random_search.best_estimator_

# Evaluate the best estimator on the test set
y_pred_test = best_estimator.predict(X_test_preprocessed)
f1_test = metrics.f1_score(y_test, y_pred_test, average='weighted')

largest = congressional_voting_results.nlargest(1, 'f1')
f1_value = largest['f1'].iloc[0]
print(f"f1 score of the best performing combination without parameter tuning: {f1_value}")
print(f"f1 score of the best estimator from RandomizedSearchCV: {f1_test}")

f1 score of the best performing combination without parameter tuning: 0.9543804053607976
f1 score of the best estimator from RandomizedSearchCV: 0.9074675324675324


### 1.3. DecisionTreeClassifier

For parameter tuning of the KNeighborsClassifier we use one of its the best combinations:
- feature_selection: p-value
- scaling: robust

In [14]:
congressional_voting = pd.read_csv("./preprocessed-datasets/CongressionVoting_prepro.csv")
congressional_voting.set_index('ID', inplace=True)
X = congressional_voting.drop("class", axis=1) 
y = congressional_voting['class'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=18) 

# Feature selection
X_train = X_train.loc[:, X_train.apply(pd.Series.nunique) != 1]
p_values = []
for feature in X_train.columns:
    f, p = chi2(X_train[[feature]], y_train)
    p_values.append(p)

p_values = np.array(p_values).reshape(-1)
p_values = pd.Series(p_values, index=X_train.columns).sort_values()
selected_features = p_values[p_values < 0.05].index

X_train_preprocessed, X_test_preprocessed = X_train[selected_features], X_test[selected_features]

scaler = RobustScaler().fit(X_train_preprocessed)
X_train_scaled = scaler.transform(X_train_preprocessed)
X_test_scaled = scaler.transform(X_test_preprocessed)

# Define the parameter grid for RandomizedSearchCV
param_grid_dt = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [2, 10, 20, 30],
    'min_samples_leaf': [1, 5, 10, 50, 100],
}

In [15]:
# Perform RandomizedSearchCV
random_search = RandomizedSearchCV(DecisionTreeClassifier(), param_distributions=param_grid_dt, scoring="f1_weighted") 
random_search.fit(X_train_scaled, y_train) 

RandomizedSearchCV(estimator=DecisionTreeClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [2, 10, 20, 30],
                                        'min_samples_leaf': [1, 5, 10, 50,
                                                             100]},
                   scoring='f1_weighted')

In [16]:
cv_results = random_search.cv_results_

results_data = {
    'criterion': cv_results['param_criterion'],
    'max_depth': cv_results['param_max_depth'],
    'min_samples_leaf': cv_results['param_min_samples_leaf'],
    'F1_score': cv_results['mean_test_score'],
    'Runtime': cv_results['mean_fit_time'] + cv_results['mean_score_time']
}


results = pd.DataFrame(data=results_data)
results.sort_values(by=['F1_score'], ascending=False, inplace=True)
results.reset_index(drop=True, inplace=True)
results

,criterion,max_depth,min_samples_leaf,F1_score,Runtime
0,gini,10,10,0.960873,0.003684
1,entropy,30,5,0.933517,0.007667
2,gini,2,1,0.933517,0.004514
3,gini,20,5,0.933517,0.003696
4,entropy,30,1,0.920617,0.004185
5,gini,10,50,0.855615,0.006490
6,gini,10,100,0.502562,0.003590
7,entropy,10,100,0.502562,0.003567
8,gini,20,100,0.502562,0.003351
9,entropy,2,100,0.502562,0.003540


In [17]:
# Get the best estimator from RandomizedSearchCV
best_estimator = random_search.best_estimator_

print(best_estimator)

# Evaluate the best estimator on the test set
y_pred_test = best_estimator.predict(X_test_scaled)
f1_test = metrics.f1_score(y_test, y_pred_test, average='weighted')

largest = congressional_voting_results.nlargest(1, 'f1')
f1_value = largest['f1'].iloc[0]
print(f"f1 score of the best performing combination without parameter tuning: {f1_value}")
print(f"f1 score of the best estimator from RandomizedSearchCV: {f1_test}")

DecisionTreeClassifier(max_depth=10, min_samples_leaf=10)
f1 score of the best performing combination without parameter tuning: 0.9543804053607976
f1 score of the best estimator from RandomizedSearchCV: 0.9696969696969697


One of the best combinations (classifier: DecisionTree, feature_selection: p-value, scaling: robust) for the congressional voting dataset had an f1 score = 0.95438. With RandomizedSearchCV we found the best parameter setting which result in a higher f1 score.  

## 2. Dataset Bank Marketing

In [18]:
bank_marketing_results = combined_results[combined_results['dataset'] == 'bank_marketing']

In [19]:
bank_marketing_results.nlargest(5, 'f1')

,dataset,classifier,splitting,feature_selection,scaling,n_features_orig,n_features,accuracy,precision,recall,f1,preprocess_time,training_time,prediction_time,f1_std
221,bank_marketing,RandomForestClassifier,cv,none,minmax,34,34,0.939190,0.939728,0.939190,0.939171,0.000012,6.995210,0.288231,0.001061
220,bank_marketing,RandomForestClassifier,holdout,none,minmax,34,34,0.937389,0.938072,0.937389,0.937368,0.000007,6.563403,0.498113,NaN
233,bank_marketing,RandomForestClassifier,cv,correlation,minmax,34,26,0.935441,0.936028,0.935441,0.935419,0.159210,7.780731,0.274147,0.001230
232,bank_marketing,RandomForestClassifier,holdout,correlation,minmax,34,26,0.933604,0.934403,0.933604,0.933577,0.149638,6.250977,0.425510,NaN
227,bank_marketing,RandomForestClassifier,cv,p-value,minmax,34,30,0.931597,0.931801,0.931597,0.931589,0.731339,9.361306,0.421076,0.000658


For parameter tuning we use the best combination:
- classifier: RandomForest
- feature_selection: None
- scaling: minmax

Besides, we also apply SMOTE on the bank marketing dataset, just like in the baseline experimentations.

In [20]:
def apply_smote(df, target_column, k_neighbors=4, random_state=321):

    X = df.drop(target_column, axis=1)
    y = df[target_column]

    sm = SMOTE(random_state=random_state, k_neighbors=k_neighbors)
    X_resampled, y_resampled = sm.fit_resample(X, y)

    return X_resampled, y_resampled

In [21]:
# Read the preprocessed dataset
bank_marketing = pd.read_csv("./preprocessed-datasets/bank_marketing_prepro.csv")
target_column = 'class'
X,y = apply_smote(bank_marketing,target_column)
  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=18) 

scaler = MinMaxScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the parameter grid for RandomizedSearchCV
param_grid = { 
    'n_estimators': [25, 50, 100, 150], 
    'max_features': ['sqrt', 'log2', None], 
    'max_depth': [10, 20, 30], 
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
} 

In [22]:
# Perform RandomizedSearchCV
random_search = RandomizedSearchCV(RandomForestClassifier(), param_distributions=param_grid, scoring="f1_weighted") 
random_search.fit(X_train_scaled, y_train) 

RandomizedSearchCV(estimator=RandomForestClassifier(),
                   param_distributions={'max_depth': [10, 20, 30],
                                        'max_features': ['sqrt', 'log2', None],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [25, 50, 100, 150]},
                   scoring='f1_weighted')

In [23]:
# Get the best estimator from RandomizedSearchCV
best_estimator = random_search.best_estimator_

# Evaluate the best estimator on the test set
y_pred_test = best_estimator.predict(X_test_scaled)
f1_test = metrics.f1_score(y_test, y_pred_test, average='weighted')

largest = bank_marketing_results.nlargest(1, 'f1')
f1_value = largest['f1'].iloc[0]
print(f"f1 score of the best performing combination without parameter tuning: {f1_value}")
print(f"f1 score of the best estimator from RandomizedSearchCV: {f1_test}")

f1 score of the best performing combination without parameter tuning: 0.9391710746130112
f1 score of the best estimator from RandomizedSearchCV: 0.9367633224597464


The search space explored by RandomizedSearchCV did not include a set of hyperparameters that performed better than the ones in the original best model.

## 3. Dataset Wine Quality

In [24]:
wine_quality_results = combined_results[combined_results['dataset'] == 'wine_quality']

In [25]:
wine_quality_results.nlargest(5, 'f1')

,dataset,classifier,splitting,feature_selection,scaling,n_features_orig,n_features,accuracy,precision,recall,f1,preprocess_time,training_time,prediction_time,f1_std
23,wine_quality,RandomForestClassifier,cv,none,minmax,13,13,0.891145,0.887836,0.891145,0.888645,0.005564,3.137737,0.063741,0.007228
41,wine_quality,RandomForestClassifier,cv,p-value,robust,13,11,0.890490,0.887290,0.890490,0.888010,0.058313,3.119370,0.066823,0.007146
11,wine_quality,RandomForestClassifier,cv,none,standard,13,13,0.889986,0.886811,0.889986,0.887407,0.006954,3.088176,0.062916,0.006602
5,wine_quality,RandomForestClassifier,cv,none,none,13,13,0.889835,0.886253,0.889835,0.887116,0.000399,3.160678,0.066656,0.006167
35,wine_quality,RandomForestClassifier,cv,p-value,standard,13,11,0.889483,0.886272,0.889483,0.887033,0.047879,3.140681,0.066032,0.006499


For parameter tuning we use the best combination:
- classifier: RandomForest
- feature_selection: None
- scaling: minmax

Besides, we also apply SMOTE on the wine quality dataset, just like in the baseline experimentations.

In [26]:
# Read the preprocessed dataset
wine_quality = pd.read_csv("./preprocessed-datasets/wine_quality_prepro.csv")
target_column = 'class'
X,y = apply_smote(wine_quality,target_column)
  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=18) 

scaler = MinMaxScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the parameter grid for RandomizedSearchCV
param_grid = { 
    'n_estimators': [25, 50, 100, 200, 500], 
    'max_features': ['sqrt', 'log2', None], 
    'max_depth': [5, 25, 50],
    'min_samples_split': [2, 5, 10]
}

In [27]:
# Perform RandomizedSearchCV
random_search = RandomizedSearchCV(RandomForestClassifier(), param_distributions=param_grid, scoring="f1_weighted") 
random_search.fit(X_train_scaled, y_train) 

RandomizedSearchCV(estimator=RandomForestClassifier(),
                   param_distributions={'max_depth': [5, 25, 50],
                                        'max_features': ['sqrt', 'log2', None],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [25, 50, 100, 200,
                                                         500]},
                   scoring='f1_weighted')

In [28]:
# Get the best estimator from RandomizedSearchCV
best_estimator = random_search.best_estimator_

# Evaluate the best estimator on the test set
y_pred_test = best_estimator.predict(X_test_scaled)
f1_test = metrics.f1_score(y_test, y_pred_test, average='weighted')

largest = wine_quality_results.nlargest(1, 'f1')
f1_value = largest['f1'].iloc[0]
print(f"f1 score of the best performing combination without parameter tuning: {f1_value}")
print(f"f1 score of the best estimator from RandomizedSearchCV: {f1_test}")

f1 score of the best performing combination without parameter tuning: 0.8886451647615082
f1 score of the best estimator from RandomizedSearchCV: 0.8645778338494036


The search space explored by RandomizedSearchCV did not include a set of hyperparameters that performed better than the ones in the original best model.

## 4. Dataset Amazon Reviews

In [29]:
reviews_results = combined_results[combined_results['dataset'] == 'reviews']

Top 5 combinations based on F1 score:

In [30]:
reviews_results.nlargest(5, 'f1')

,dataset,classifier,splitting,feature_selection,scaling,n_features_orig,n_features,accuracy,precision,recall,f1,preprocess_time,training_time,prediction_time,f1_std
154,reviews,RandomForestClassifier,holdout,none,standard,10000,10000,0.533333,0.633823,0.533333,0.524547,1.145058,1.584533,0.050707,NaN
185,reviews,RandomForestClassifier,cv,p-value,robust,10000,6054,0.526667,0.581471,0.526667,0.511494,73.258305,1.383942,0.028825,0.051475
184,reviews,RandomForestClassifier,holdout,p-value,robust,10000,5797,0.520000,0.604660,0.520000,0.508507,68.972446,1.184847,0.032580,NaN
191,reviews,RandomForestClassifier,cv,p-value,minmax,10000,6054,0.522667,0.584946,0.522667,0.507046,73.231496,1.372783,0.027929,0.054543
179,reviews,RandomForestClassifier,cv,p-value,standard,10000,6054,0.521333,0.592879,0.521333,0.506898,72.612922,1.398013,0.029681,0.038711


For parameter tuning we use the best combination:
- classifier: RandomForest
- feature_selection: None
- scaling: standard

In [31]:
# Read the preprocessed dataset
reviews = pd.read_csv("./preprocessed-datasets/Review_prepro.csv")
X = reviews.drop("class", axis=1) 
y = reviews['class'] 
  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=18) 

scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the parameter grid for RandomizedSearchCV
param_grid = { 
    'n_estimators': [25, 50, 100, 150], 
    'max_features': ['sqrt', 'log2', None], 
    'max_depth': [10, 20, 30], 
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
} 

In [32]:
# Perform RandomizedSearchCV
random_search = RandomizedSearchCV(RandomForestClassifier(), param_distributions=param_grid, scoring="f1_weighted") 
random_search.fit(X_train_scaled, y_train) 

/usr/local/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


RandomizedSearchCV(estimator=RandomForestClassifier(),
                   param_distributions={'max_depth': [10, 20, 30],
                                        'max_features': ['sqrt', 'log2', None],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [25, 50, 100, 150]},
                   scoring='f1_weighted')

In [33]:
# Get the best estimator from RandomizedSearchCV
best_estimator = random_search.best_estimator_

# Evaluate the best estimator on the test set
y_pred_test = best_estimator.predict(X_test_scaled)
f1_test = metrics.f1_score(y_test, y_pred_test, average='weighted')

largest = reviews_results.nlargest(1, 'f1')
f1_value = largest['f1'].iloc[0]
print(f"f1 score of the best performing combination without parameter tuning: {f1_value}")
print(f"f1 score of the best estimator from RandomizedSearchCV: {f1_test}")

f1 score of the best performing combination without parameter tuning: 0.5245474278807613
f1 score of the best estimator from RandomizedSearchCV: 0.5209333295607806
